In [1]:
!pip install cdsapi

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for cdsapi: filename=cdsapi-0.6.1-py2.py3-none-any.whl size=12009 sha256=c4214562a0018751568925a194ea568599567f1dd12dec470457d27e942318d3
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7c/63/08/45461d6f6636c1aba7846828d8c787a064073945048f76d44a
Successfully built cdsapi


In [7]:
import cdsapi
import os
from zipfile import ZipFile
import glob
import boto3
import shutil


In [8]:
key="187933:64547c55-b131-4cec-a39e-f455bf2b12b3"
url = "https://cds.climate.copernicus.eu/api/v2"
user = "129570"
cds =cdsapi.Client(key=key,url=url)
bucket = "climate-action-datalake"
localDownloadPath = "data/"
zipName="sample.zip"
datalakeZone="landing"
source="agera5"
# print(key)
s3_client = boto3.client('s3')
historicalYears = range(2023,2024)
months = range(1,13)
variableNames = ["2m_relative_humidity"] # 2m_relative_humidity-solar_radiation_flux-10m_wind_speed
days=range(1,32)
statistics = ['24_hour_maximum', '24_hour_mean', '24_hour_minimum']
variableS3= "2m_relative_humidity" #  2m_relative_humidity-10m-wind-speed-solar-radiation-flux


In [9]:
def downloadAgera5Data(variableName:str
                       ,statistics:[]
                       ,year:[]
                       ,month:[]
                       ,day:[]
                       ,bucket:str
                       ,downloadPath:str
                       ,zipName:str
                       ,datalakeZone:str
                       ,source:str
                       ,variable:str
                       ,url:str
                       ,key:str
                      ):
    cds =cdsapi.Client(key=key,url=url)
    sisK="sis-agrometeorological-indicators"
    for _year in year:
        for _month in month:
            for _day in day:
                jsonCopernicus = {'format':'zip'
                                   ,'version': '1_1'
                            #  ,'statistic':statistics
                                ,'variable': variableName
                                ,'year':[str(_year)]
                                ,'month':[f"{_month:02d}"]
                                ,'day':[f"{_day:02d}"]  
                                ,'time': '12_00'
                              }
                try:

                    cds.retrieve(sisK,jsonCopernicus,localDownloadPath+zipName)
                    with  ZipFile(downloadPath+zipName,'r') as z:
                        z.extractall(path=downloadPath)
                    filesUpload = glob.glob(downloadPath+"*.nc")
                    for _file in filesUpload:
                        s3Path= f"zone={datalakeZone}/source={source}/variable={variable}/"
                        response = s3_client.upload_file(_file
                                                         , bucket
                                                         ,s3Path+_file.split("/")[1])
                        os.remove(_file)
                        
                except:
                    print(_year,_month,_day)

In [ ]:
downloadAgera5Data(variableName=variableNames
                   ,statistics=statistics
                   ,year=historicalYears
                   ,month=months
                   ,day=days
                   ,bucket=bucket
                   ,downloadPath=localDownloadPath
                   ,zipName=zipName
                   ,datalakeZone=datalakeZone
                   ,source=source
                   ,variable=variableS3
                   ,url=url
                   ,key=key
                  )

In [ ]:
filesUpload = glob.glob(localDownloadPath+"*.nc")

In [ ]:
len(filesUpload)

In [ ]:
# os.remove(filesUpload[1])